In [8]:
# NOTES

# Mind map
# https://miro.com/app/board/o9J_kwo5k8s=/

# Prediction labels
# [       0     ,      1    ,   2  ]
# ["pedestrian" , "bicycle" , "car"]

# Coordinate format: [longitude, latitude]

# Time format used:
# "%Y-%m-%dT%H:%M:%S.%fZ"

# Sequence format: Geojson
#     type
#     properties:
#       captured_at (start of sequence)
#       coordinateProperties
#         cas (camera angle between 0 and 360)
#         image_keys
#     geometry
#       coordinates [longitude, latitude]

# Image format: Geojson
#    type
#    properties:
#       captured_at, 
#       camera_model, camera_make, ca, pano, seq_key, key, username, user_key
#    geometry
    

# Potential features
#     Average velocity
#     Maximum velocity
#     Maximum stop time (calculate time between neighbours points relative to avg speed)
#     Possible to use CAS coordinate property to estimte slope and the difference in velocity?
#     Smallest turn (small turn will indicate not a car road)
#     Trajectory measurements
#     length
#     
#     
#     Image detection
#       Traffic lights (number or boolean)
#       Roundabouts (number or boolean)
#       Bicycle tracks on the side
#       Highway/number or lanes
#       Sidewalk

# Features columns
# 0 =label, 1 = seq_id, 2 = avg_speed(m/s) 

# Imports

In [181]:
# Imports
import json
import math
import numpy as np
from datetime import datetime, date
from pprint import pprint as pp


# Utility functions

In [182]:
# Utility functions
def readJson(file):
    with open(file,"r") as f:
        rf=json.load(f)
    return rf

def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()



# Feature functions

In [183]:
# Functions
def avg_speed_n_forward(sequence, start,n):
    return 0

def height_calculations(images):
    request = ""
    data = []
    print(images)
    for im in images:
        data.append[im]
    return 0,0

In [207]:
class Sequence:
    def __init__(self, sequence):
        #{type:"FeatureCollection",features:[{id,images},{id,images},...]
        self.id = sequence["id"]
        self.images = sequence["images"]
        self.iterate()
    
    def iterate(self):
        if(len(self.images)==0):
            return 0
        dist = duration = max_speed = 0
        prev = self.images[0]
        
        for feat in self.images[1:]:
            feat_dist = distance(prev["geometry"]["coordinates"], feat["geometry"]["coordinates"])
            feat_time = datetime.strptime(feat["properties"]["captured_at"], "%Y-%m-%dT%H:%M:%S.%fZ")
            prev_time = datetime.strptime(prev["properties"]["captured_at"], "%Y-%m-%dT%H:%M:%S.%fZ")
            feat_dur = abs(feat_time-prev_time).total_seconds()
            feat_speed = 0 if feat_dur==0.0 else feat_dist/feat_dur
            
            if(feat_speed>max_speed):
                max_speed = feat_speed
            dist+=feat_dist
            duration+=feat_dur
            prev = feat
        
        self.avg_speed = 0 if duration==0.0 else dist/duration
        self.max_speed = max_speed
        self.distance = dist
        self.duration = duration
    
    def get_features(self):
        features = vars(self)
        excluded_keys = {"images"}
        return [features[x] for x in features if x not in excluded_keys]
    
    @staticmethod
    def distance(start, end):
        start = [deg2rad(start[0]),deg2rad(start[1])]
        end = [deg2rad(end[0]),deg2rad(end[1])]

        R =  6373000
        lon1, lat1, lon2, lat2 = start[0], start[1], end[0], end[1]
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = (math.sin(dlat/2)**2)+math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
        c = 2*math.atan2(math.sqrt(a), math.sqrt(1-a))
        return R*c
    
    @staticmethod
    def deg2rad(deg):
        return deg*(math.pi/180)
    

In [208]:
# write features for an area
def area_features(data, write=False, stop_early=False, sample_size=20):
    num_seq = len(data["features"])
    print(f"[INFO] {num_seq} sequences:")
    f = open("training_data/features.csv", "w+")
    for num, seq in enumerate(data["features"]):
        if(stop_early==True and num>sample_size-1):
            break
        printProgressBar(num, num_seq)
        seq = Sequence(seq)
        if(len(seq.images) < 2):
            continue
#         avg_height_diff, max_height_diff = height_calculations(images)
        
#         if(dist==0 or avg_speed==0):
#             continue
        if(write==True):
            s = ""
            features = seq.get_features()
            for feat in features[1:]:
                s += f"{feat},"
            f.write(f"{s}{features[0]}\n")
    f.close()
    print("[INFO] Done generating features")

In [199]:
data = readJson("mapillary_data/mapillary_data.json")

In [200]:
self = Sequence(data["features"][2140])

In [209]:
area_features(data, stop_early=False, write=True)

[INFO] 3039 sequences:
[INFO] Done generating features██████████████████████████████████████████████████████████████████████-| 100.0% 
